In [1]:
import torch
import pandas as pd
import os

In [14]:
class TitlePartyModel(torch.nn.Module):
    def __init__(self):
        super(TitlePartyModel, self).__init__()
        self.input = torch.nn.Linear(2048,2048, dtype=torch.float32)
        self.input_activation = torch.nn.Sigmoid()
        self.hidden1 = torch.nn.Linear(2048,2*2048)
        self.hidden1_activation = torch.nn.Sigmoid()
        self.hidden2 = torch.nn.Linear(2*2048,128)
        self.hidden2_activation = torch.nn.Sigmoid()
        # 4 political party choices
        self.hidden3 = torch.nn.Linear(128,4)
        self.output = torch.nn.Softmax()

    def forward(self, x):
        x = self.input(x)
        x = self.input_activation(x)
        x = self.hidden1(x)
        x = self.hidden1_activation(x)
        x = self.hidden2(x)
        x = self.hidden2_activation(x)
        x = self.hidden3(x)
        x = self.output(x)
        return x



In [3]:
title_model = TitlePartyModel()
print("the model")
print(title_model)
print("just the 2nd layer")
print(title_model.hidden1)
print("parameters")
for p in title_model.parameters():
    print(p)
print("2nd layer params")
for p in title_model.hidden1.parameters():
    print(p)

the model
TitlePartyModel(
  (input): Linear(in_features=2048, out_features=2048, bias=True)
  (input_activation): Sigmoid()
  (hidden1): Linear(in_features=2048, out_features=4096, bias=True)
  (hidden1_activation): Sigmoid()
  (hidden2): Linear(in_features=4096, out_features=128, bias=True)
  (hidden2_activation): Sigmoid()
  (hidden3): Linear(in_features=128, out_features=4, bias=True)
  (output): Softmax(dim=None)
)
just the 2nd layer
Linear(in_features=2048, out_features=4096, bias=True)
parameters
Parameter containing:
tensor([[ 0.0096, -0.0070,  0.0135,  ..., -0.0206, -0.0211, -0.0002],
        [ 0.0089, -0.0115,  0.0043,  ..., -0.0040, -0.0093, -0.0029],
        [-0.0154, -0.0010,  0.0078,  ..., -0.0210, -0.0059, -0.0199],
        ...,
        [ 0.0219, -0.0059, -0.0082,  ..., -0.0195,  0.0005,  0.0014],
        [-0.0078,  0.0055, -0.0177,  ...,  0.0065,  0.0131,  0.0144],
        [ 0.0141,  0.0005,  0.0211,  ...,  0.0064, -0.0143, -0.0090]],
       requires_grad=True)
Paramete

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import fnmatch
import numpy as np

In [5]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(title_model.parameters(), lr=0.001, momentum=0.9)

In [6]:
train_files = []
test_files = []
split = 0.7
token_path = os.path.join(os.getcwd(),"..","data","tokenized")
print(token_path)
for root, dirs, files in os.walk(token_path):
    for f in files:
        if fnmatch.fnmatch(f, "*shrunk*"):
            if np.random.sample(1) <= split:
                print(f'train: {f}')
                train_files.append(os.path.join(root, f))
            else:
                print(f'test : {f}')
                test_files.append(os.path.join(root, f))
print(train_files)
print(test_files)

C:\Users\benja\git-projects\bitbucket\nlp_legislation_prediction\training\..\data\tokenized
train: summary_bill_1811_1393180-shrunk.pkl
train: summary_bill_1811_1393181-shrunk.pkl
train: summary_bill_1811_1470063-shrunk.pkl
train: summary_bill_1811_1506887-shrunk.pkl
test : summary_bill_1959_1542899-shrunk.pkl
train: summary_bill_1959_1545862-shrunk.pkl
train: summary_bill_1959_1546074-shrunk.pkl
test : summary_bill_1959_1546096-shrunk.pkl
['C:\\Users\\benja\\git-projects\\bitbucket\\nlp_legislation_prediction\\training\\..\\data\\tokenized\\summary_bill_1811_1393180-shrunk.pkl', 'C:\\Users\\benja\\git-projects\\bitbucket\\nlp_legislation_prediction\\training\\..\\data\\tokenized\\summary_bill_1811_1393181-shrunk.pkl', 'C:\\Users\\benja\\git-projects\\bitbucket\\nlp_legislation_prediction\\training\\..\\data\\tokenized\\summary_bill_1811_1470063-shrunk.pkl', 'C:\\Users\\benja\\git-projects\\bitbucket\\nlp_legislation_prediction\\training\\..\\data\\tokenized\\summary_bill_1811_1506887-

In [21]:
class SummaryDataSet(torch.utils.data.Dataset):
    def __init__(self, file_path_arr):
        self.data_frames = []
        for f in file_path_arr:
            print(f"loading {f}")
            self.data_frames.append(pd.read_pickle(f, compression="gzip"))

    def __len__(self):
        return len(self.data_frames)

    def __getitem__(self, idx):
        next_df = self.data_frames[idx]
        party = next_df["party"][0] # they are all the same, so just pick the first one
        encoding = torch.tensor(np.array(next_df["input_shrunk"]),dtype=torch.float)
        # 4 politcal party choices
        party_arr = np.zeros(4,dtype=int)
        # the party index was stored as value with a starting index of 1 -- rethink this
        party_arr[party-1] = 1 # set the value to 1 for the party index
        return encoding, torch.tensor(party_arr,dtype=torch.float)

In [8]:
from torch.utils.data import DataLoader
train_data_set = SummaryDataSet(train_files)
test_data_set = SummaryDataSet(test_files)
train_loader = DataLoader(train_data_set,batch_size=1,shuffle=True)
test_loader = DataLoader(test_data_set, batch_size=1, shuffle=True)

loading C:\Users\benja\git-projects\bitbucket\nlp_legislation_prediction\training\..\data\tokenized\summary_bill_1811_1393180-shrunk.pkl
loading C:\Users\benja\git-projects\bitbucket\nlp_legislation_prediction\training\..\data\tokenized\summary_bill_1811_1393181-shrunk.pkl
loading C:\Users\benja\git-projects\bitbucket\nlp_legislation_prediction\training\..\data\tokenized\summary_bill_1811_1470063-shrunk.pkl
loading C:\Users\benja\git-projects\bitbucket\nlp_legislation_prediction\training\..\data\tokenized\summary_bill_1811_1506887-shrunk.pkl
loading C:\Users\benja\git-projects\bitbucket\nlp_legislation_prediction\training\..\data\tokenized\summary_bill_1959_1545862-shrunk.pkl
loading C:\Users\benja\git-projects\bitbucket\nlp_legislation_prediction\training\..\data\tokenized\summary_bill_1959_1546074-shrunk.pkl
loading C:\Users\benja\git-projects\bitbucket\nlp_legislation_prediction\training\..\data\tokenized\summary_bill_1959_1542899-shrunk.pkl
loading C:\Users\benja\git-projects\bitbu

In [31]:
def train_one_epoch(epoch_idx, model, summary_writer):
    running_loss = 0.
    last_loss = 0.
    for i, data in enumerate(train_loader):
        inputs, label = data
        input_tensor = inputs.view(1,-1)
        optimizer.zero_grad()
        outputs = model(input_tensor)
        loss = loss_fn(outputs, label)
        print(loss)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        last_loss = running_loss
        summary_idx = i * len(train_loader) + i + 1
        summary_writer.add_scalar("loss/train", last_loss, summary_idx)

    return last_loss

In [32]:
import datetime
from torch.utils.tensorboard import SummaryWriter
start_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
writer = SummaryWriter('runs/fashion_trainer_{}'.format(start_time))
EPOCHS = 10
epoch_num = 0
title_model = TitlePartyModel()
losses = []
for epoch in range(EPOCHS):
    print("turn on training")
    title_model.train(True)
    print("running one epoch")
    last_epoch_loss = train_one_epoch(epoch_num,title_model, writer)
    print("turn off training")
    print(f'epoch loss {last_epoch_loss}')

    title_model.train(False)

    running_validation_loss = 0.0
    for i, vdata in enumerate(test_loader):
        vinputs, vlabel = vdata
        print(f"label : {vlabel}")
        voutputs = title_model(vinputs)
        print(f"vOutput: {voutputs}")
        vloss = loss_fn(voutputs, vlabel)
        running_validation_loss += vloss

    avg_vloss = running_validation_loss / len(test_loader)
    losses.append(avg_vloss)
    print('LOSS train {} valid {}'.format(last_epoch_loss, avg_vloss))
#    writer.add_scalars("Training vs Valiation loss",{"training": last_epoch_loss, "validation": avg_vloss}, epoch_num+1)
#    writer.flush()
    epoch_num += 1

turn on training
running one epoch
      status  party  input_shrunk
0          5      1      0.411736
1          5      1      0.019537
2          5      1      0.313383
3          5      1     -0.088169
4          5      1     -0.271366
...      ...    ...           ...
2043       5      1     -0.026054
2044       5      1     -0.318737
2045       5      1      0.723295
2046       5      1      0.055131
2047       5      1     -0.453075

[2048 rows x 3 columns]
tensor(1.3541, grad_fn=<DivBackward1>)
      status  party  input_shrunk
0          2      1      0.183673
1          2      1      0.244852
2          2      1     -0.095650
3          2      1     -0.322488
4          2      1      0.048414
...      ...    ...           ...
2043       2      1      0.808546
2044       2      1     -0.472243
2045       2      1      0.261753
2046       2      1     -0.975441
2047       2      1     -1.050877

[2048 rows x 3 columns]
tensor(1.3538, grad_fn=<DivBackward1>)
      status  party  

C:\Users\benja\AppData\Local\Temp\ipykernel_25856\1004825444.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  party_arr = np.zeros(4,dtype=np.int)
C:\Users\benja\AppData\Local\Temp\ipykernel_25856\685939361.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output(x)
C:\Users\benja\AppData\Local\Temp\ipykernel_25856\1004825444.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doin

      status  party  input_shrunk
0          4      1      0.169372
1          4      1      0.090074
2          4      1     -0.160564
3          4      1      0.131678
4          4      1      0.023742
...      ...    ...           ...
2043       4      1     -0.011668
2044       4      1      0.040745
2045       4      1     -1.896426
2046       4      1     -0.620909
2047       4      1     -1.108641

[2048 rows x 3 columns]
tensor(1.3538, grad_fn=<DivBackward1>)
      status  party  input_shrunk
0          3      4      0.254739
1          3      4     -0.195609
2          3      4      0.126331
3          3      4      0.262518
4          3      4      0.010650
...      ...    ...           ...
2043       3      4      0.251763
2044       3      4      0.957258
2045       3      4     -0.232392
2046       3      4     -0.157064
2047       3      4     -0.443530

[2048 rows x 3 columns]
tensor(1.4283, grad_fn=<DivBackward1>)
      status  party  input_shrunk
0          2      1   

C:\Users\benja\AppData\Local\Temp\ipykernel_25856\1004825444.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  party_arr = np.zeros(4,dtype=np.int)
C:\Users\benja\AppData\Local\Temp\ipykernel_25856\685939361.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output(x)
C:\Users\benja\AppData\Local\Temp\ipykernel_25856\1004825444.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doin

      status  party  input_shrunk
0          3      4      0.081985
1          3      4      0.137535
2          3      4     -0.133649
3          3      4      0.125648
4          3      4      0.155327
...      ...    ...           ...
2043       3      4     -0.287244
2044       3      4      0.133067
2045       3      4     -0.510854
2046       3      4     -0.383557
2047       3      4     -1.129584

[2048 rows x 3 columns]
tensor(1.4283, grad_fn=<DivBackward1>)
      status  party  input_shrunk
0          1      4      0.005786
1          1      4     -0.121680
2          1      4      0.401604
3          1      4     -0.332910
4          1      4     -0.551901
...      ...    ...           ...
2043       1      4      0.616539
2044       1      4      0.325676
2045       1      4     -0.301177
2046       1      4     -0.332650
2047       1      4     -1.416497

[2048 rows x 3 columns]
tensor(1.4280, grad_fn=<DivBackward1>)
      status  party  input_shrunk
0          4      1   

...

C:\Users\benja\AppData\Local\Temp\ipykernel_25856\1004825444.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  party_arr = np.zeros(4,dtype=np.int)
C:\Users\benja\AppData\Local\Temp\ipykernel_25856\685939361.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output(x)
C:\Users\benja\AppData\Local\Temp\ipykernel_25856\1004825444.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doin

      status  party  input_shrunk
0          3      4      0.081985
1          3      4      0.137535
2          3      4     -0.133649
3          3      4      0.125648
4          3      4      0.155327
...      ...    ...           ...
2043       3      4     -0.287244
2044       3      4      0.133067
2045       3      4     -0.510854
2046       3      4     -0.383557
2047       3      4     -1.129584

[2048 rows x 3 columns]
tensor(1.4283, grad_fn=<DivBackward1>)
turn off training
epoch loss 8.346179842948914
      status  party  input_shrunk
0          5      1      0.198884
1          5      1     -0.333843
2          5      1     -0.513041
3          5      1      0.245438
4          5      1      0.287163
...      ...    ...           ...
2043       5      1      0.432693
2044       5      1     -0.518468
2045       5      1     -0.271924
2046       5      1     -0.210291
2047       5      1     -0.845959

[2048 rows x 3 columns]
label : tensor([[1., 0., 0., 0.]])
vOutput: tens

C:\Users\benja\AppData\Local\Temp\ipykernel_25856\1004825444.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  party_arr = np.zeros(4,dtype=np.int)
C:\Users\benja\AppData\Local\Temp\ipykernel_25856\685939361.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output(x)
C:\Users\benja\AppData\Local\Temp\ipykernel_25856\1004825444.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doin

turn off training
epoch loss 8.346179842948914
      status  party  input_shrunk
0          5      1      0.198884
1          5      1     -0.333843
2          5      1     -0.513041
3          5      1      0.245438
4          5      1      0.287163
...      ...    ...           ...
2043       5      1      0.432693
2044       5      1     -0.518468
2045       5      1     -0.271924
2046       5      1     -0.210291
2047       5      1     -0.845959

[2048 rows x 3 columns]
label : tensor([[1., 0., 0., 0.]])
vOutput: tensor([[0.2831, 0.2635, 0.2450, 0.2084]], grad_fn=<SoftmaxBackward0>)
      status  party  input_shrunk
0          1      1      0.012870
1          1      1     -0.034266
2          1      1      0.075810
3          1      1      0.428671
4          1      1      0.167113
...      ...    ...           ...
2043       1      1      0.735692
2044       1      1      1.607339
2045       1      1      0.314064
2046       1      1     -0.266179
2047       1      1     -0.4645

C:\Users\benja\AppData\Local\Temp\ipykernel_25856\1004825444.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  party_arr = np.zeros(4,dtype=np.int)
C:\Users\benja\AppData\Local\Temp\ipykernel_25856\685939361.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output(x)
C:\Users\benja\AppData\Local\Temp\ipykernel_25856\1004825444.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doin

In [27]:
print(losses)

[tensor(1.4816, grad_fn=<DivBackward0>), tensor(1.4816, grad_fn=<DivBackward0>), tensor(1.4816, grad_fn=<DivBackward0>), tensor(1.4816, grad_fn=<DivBackward0>), tensor(1.4816, grad_fn=<DivBackward0>), tensor(1.4816, grad_fn=<DivBackward0>), tensor(1.4816, grad_fn=<DivBackward0>), tensor(1.4816, grad_fn=<DivBackward0>), tensor(1.4816, grad_fn=<DivBackward0>), tensor(1.4816, grad_fn=<DivBackward0>)]
